In [1]:
# DO NOT modify this cell. 
filename = "Buchtel.pgm"
verticalSeam2Remove = 10
horizontalSeam2Remove = 8

# Yes, you can create your own test cases and you should. Do it in a new cell.

In [2]:
# Part I : Vertical seam removal only
import numpy as np

# function to read PGM file and extract image data
# return the image data as a numpy array
def read_pgm(file_name):
    # initialize the varible to store the data of pgm image
    width = 0
    height = 0
    max_val = 0
    image_data = None
    
    try:
        with open(file_name, 'r') as f:
            # read the first line containing the magic number('P2')
            magic_number = f.readline().strip()
            if magic_number != 'P2':
                print("Invalid PGM file format. Magic number ,ust be 'P2'")
                return None
            
            # Read the second line which may contain comments
            second_line = f.readline().strip()
                
            # read the third line for widht and height
            width, height = map(int, f.readline().split())
            
            # read the fourth line as maximum grayscale value
            max_val = int(f.readline())
            
            #read the remaining line containing the grayscale values
            # convert the value to integers and store them in list
            image_data = []
            for line in f:
                image_data.extend(map(int, line.split()))
    except FileNotFoundError:
        print("File not found:", file_name)
        return None
    except Exception as e:
        print("An error occured while reading the file:", e)
        return None
    
    #convert the list of grayscale values to a numpy array of the appropraite shape
    image_data = np.array(image_data).reshape(height, width)
    
    return image_data
    
# function to write data to a pgm output file
def write_pgm(file_name, image_data):
    try:
        #open the file for writing
        with open(file_name, 'w') as f:
            # write the pgm header
            f.write("P2\n") #magic numnber
            f.write("# Created by IrfanView\n") #comment
            f.write("{} {}\n".format(image_data.shape[1], image_data.shape[0])) #width and height
            f.write("255\n") #maximum frayscale value
            
            
            #write the image data
            for row in image_data:
                for value in row:
                    f.write(str(value) + " ")
                f.write("\n")
    except Exception as e:
        print("An error occured while writing file:", e)

# function to compute the energy of each pixel in the image
def compute_energy(image):
    # get the enegry of each pixel in the image
    
    # get the dimension of the image
    height, width = image.shape
    
    # pad the image boundry wiht the neasrest pixel values
    padded_image = np.pad(image, ((1, 1), (1, 1)), mode='edge')
    
    # initialize an array to store the energy value
    energy = np.zeros_like(image, dtype=np.int32)
    
    #compute the energy for each pixel using the gradient magnitude forumla
    for i in range(height):
        for j in range(width):
            # compute the horizintal gradianet (gx)
            gx = abs(int(padded_image[i + 1, j + 1]) - int(padded_image[i + 1, j])) + abs(int(padded_image[i + 1, j + 1]) - int(padded_image[i + 1, j + 2]))
            
            #compute the vertical gradirent (gx)
            gy = abs(int(padded_image[i + 1, j + 1]) - int(padded_image[i, j + 1])) + abs(int(padded_image[i + 1, j + 1]) - int(padded_image[i + 2, j + 1]))
            
            # compute the enegry using the gradient magnitude formula
            energy[i, j] = gx + gy
    
    return energy

# function to compute the cumulative energy
def compute_cumulative_minimum_energy(energy):
    # Get the dimensions of the energy matrix
    height, width = energy.shape
    
    # Initialize an array to store the cumulative minimum energy
    cumulative_energy = np.zeros_like(energy, dtype=np.int32)
    
    # Copy the first row of the energy matrix to the cumulative energy matrix
    cumulative_energy[0] = energy[0]
    
    # Iterate over the rows of the energy matrix from the second row
    for i in range(1, height):
        # Compute the cumulative minimum energy for each pixel in the current row
        for j in range(width):
            # Find the minimum cumulative energy of the neighboring pixels in the previous row
            min_prev_energy = min(cumulative_energy[i - 1, max(j - 1, 0):min(j + 2, width)])
            # Add the current pixel's energy to the minimum cumulative energy
            cumulative_energy[i, j] = energy[i, j] + min_prev_energy
    
    return cumulative_energy

# find vertical seam with the lowest enegry
def find_vertical_seam(cumulative_energy):
    # Get the dimensions of the cumulative energy matrix
    height, width = cumulative_energy.shape
    
    # Initialize an array to store the vertical seam
    vertical_seam = np.zeros(height, dtype=np.int32)
    
    # Find the pixel with the minimum cumulative energy in the bottom row
    min_energy_index = np.argmin(cumulative_energy[-1])
    vertical_seam[-1] = min_energy_index
    
    # Trace the path of minimum cumulative energy upwards to find the vertical seam
    for i in range(height - 2, -1, -1):
        # Find the column index of the pixel with minimum cumulative energy in the previous row
        j = vertical_seam[i + 1]
        # Get the indices of neighboring pixels in the previous row
        neighbors_indices = [max(j - 1, 0), j, min(j + 1, width - 1)]
        # Find the index of the neighbor with minimum cumulative energy
        min_energy_index = neighbors_indices[np.argmin(cumulative_energy[i, neighbors_indices])]
        
        # Handle tie-breaking by selecting the leftmost pixel among those with the same minimum energy
        if cumulative_energy[i, min_energy_index] == cumulative_energy[i, j]:
            min_energy_index = min(j, min_energy_index)
        
        # Store the column index of the pixel with minimum cumulative energy in the vertical seam
        vertical_seam[i] = min_energy_index
    
    return vertical_seam

# function to remove the vertical seam
def remove_vertical_seam(image, seam):
    # Get the dimensions of the original image
    height, width = image.shape
    
    # Create a new image with one fewer column
    new_image = np.zeros((height, width - 1), dtype=np.int32)
    
    # Remove the pixels along the seam from the original image
    for i in range(height):
        # Ensure the seam index is within bounds
        seam[i] = max(0, min(seam[i], width - 1))
        # Copy the pixels before the seam
        new_image[i, :seam[i]] = image[i, :seam[i]]
        # Copy the pixels after the seam
        new_image[i, seam[i]:] = image[i, seam[i] + 1:]
    
    return new_image

    
# function to perform seam curving
def seam_carivng(image_file, verticalSeam2Remove):
    # read the input image
    image = read_pgm(image_file)
    
    #repeat the seam removal process for the specified number of times
    for _ in range(verticalSeam2Remove):
        # compute the energy of each pixel
        energy = compute_energy(image)
        
        # compute cumulative energy matrix 
        cumulative_energy = compute_cumulative_minimum_energy(energy)
        
        # find the vertical seam from with the lowest energy
        seam = find_vertical_seam(cumulative_energy)
        
        # remove the vertical seam from the image
        image = remove_vertical_seam(image, seam)
        
    # Save your processed file to img_processed_v_h.pgm
    filename1 = image_file.split(".")[0]+"_processed_"+str(verticalSeam2Remove)+"_0.pgm"
    print(filename1)
    write_pgm(filename1, image)
    
# calling the function to perform seam curving
seam_carivng(filename, verticalSeam2Remove)

Buchtel_processed_10_0.pgm


In [3]:
# Part II : both vertical and horizontal seams removal 

# function to perform seam curving both vertical and horizontal seam removal
# first complete the vertical seam removal and then horixzontal seam removal
def seam_carving_vh(image_file, num_of_vertival_seam, num_of_horizontal_seam):
    #read the input image
    image = read_pgm(image_file)

    #repeat the vertical seam removal process for the specified number of times
    for _ in range(num_of_vertival_seam):
        # compute the energy of each pixel
        energy = compute_energy(image)
        
        # compute cumulative energy matrix
        cumulative_energy = compute_cumulative_minimum_energy(energy)
        
        # find the vertical seam
        seam = find_vertical_seam(cumulative_energy)
        
        # remove the vertical seam from the image
        image = remove_vertical_seam(image, seam)
    
    #transpose the image for horizonatal seam to use with vertical seam
    image = np.transpose(image)
    
    #repeat the horizontal seam removal process for the specified number of times
    for _ in range(num_of_horizontal_seam):
        # compute the energy of each pixel
        energy = compute_energy(image)
        
        # compute the cumulative energy matrix
        comulative_energy = compute_cumulative_minimum_energy(energy)
        
        # find the vertical seam
        seam = find_vertical_seam(comulative_energy)
        
        # remove the vertical seam from the image
        image = remove_vertical_seam(image, seam)
    
    # again transpose the image to get the original one
    image = np.transpose(image)
    # Save your processed file to img_processed_v_h.pgm
    filename2 = image_file.split(".")[0]+"_processed_"+ \
                str(verticalSeam2Remove)+"_"+str(horizontalSeam2Remove)+".pgm"
    print(filename2)
    write_pgm(filename2, image)

seam_carving_vh(filename, verticalSeam2Remove, horizontalSeam2Remove)

Buchtel_processed_10_8.pgm


In [4]:
# This is for graduate students only
# filenameG = "flower.ppm"
# vSeam2Remove = 20
# hSeam2Remove = 25

In [5]:
# This is for graduate students only
# Part III : color img in ppm
# filenameG2 = filenameG.split(".")[0]+"_processed_"+ \
#             str(vSeam2Remove)+"_"+str(hSeam2Remove)+".ppm"
# print(filenameG2)
# your code, add cells as you need

flower_processed_20_25.ppm
